In [1]:
!head -3 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-19143

['2013-12-28 23:40:34', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1087', '3', '3', '8', '2148', '-1']
['2013-12-28 23:41:41', '8613770773315', '111.30.132.38', 'web', 'wspeed.qq.com', 'qq', '-', '1621', '5', '5', '8', '2148', '-1']
['2013-12-28 23:43:52', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1140', '5', '5', '8', '2148', '-1']


In [1]:
# 过滤出需要用户和字段的三天数据
qq_5000_user_dpi_day1_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-*"
qq_5000_user_dpi_day2_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131229/part-*"
qq_5000_user_dpi_day3_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131230/part-*"

qq_5000_user_dpi_day4_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/20131231/part-*"

qq_5000_user_dpi_3days_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/*/part-*"

In [2]:
qq_5000_user_dpi_day1 = sc.textFile(qq_5000_user_dpi_day1_f).coalesce(50)  # repartition
qq_5000_user_dpi_day2 = sc.textFile(qq_5000_user_dpi_day2_f).coalesce(50)
qq_5000_user_dpi_day3 = sc.textFile(qq_5000_user_dpi_day3_f).coalesce(50)

qq_5000_user_dpi_day4 = sc.textFile(qq_5000_user_dpi_day4_f).coalesce(50)

qq_5000_user_dpi = sc.textFile(qq_5000_user_dpi_3days_f).coalesce(50)  

In [3]:
qq_5000_user_dpi_day4.count()

519545

In [4]:
qq_5000_user_dpi.count()

1670898

In [5]:
1670898+519545

2190443

## qq_user_node

### qq-server-ip

In [4]:
qq_server = qq_5000_user_dpi.map(lambda x: (eval(x)[2],1)).groupByKey().mapValues(len) 

In [5]:
qq_server_ip = qq_server.map(lambda x: x[0])

In [6]:
qq_server_ip.count()  # 共有1684个server ip

1684

In [7]:
qq_server_ip.take(3)

['14.17.19.77', '111.10.27.180', '111.30.132.109']

In [10]:
# 将1684个qq server ip保存到文件
# '14.17.19.77'
qq_server_ip.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_ip_1684")

### qq-user-brand

In [77]:
# 5000个user的终端品牌。('8613851429132', {'������'})  # 总共有62个终端品牌
user_brand_set = qq_5000_user_dpi.map(lambda x: (eval(x)[1], eval(x)[12])).groupByKey().mapValues(set)n

In [78]:
# 有67个用户存在两个以上终端(####取了其中一个作为其终端品牌？####)。('8613851429132', {'������'})
user_brand = user_brand_set.map(lambda x: (x[0],list(x[1])[-1]))

In [80]:
# 将每个用户及其终端品牌保存到文件。
# ('8613851429132', '������')
user_brand.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_brand")

### user_active_time

In [74]:
user_active_time_3days = qq_5000_user_dpi.map(lambda x: (eval(x)[1],eval(x)[0][-8:-6])).groupByKey().mapValues(list)

In [75]:
print(user_active_time_3days.count())
print(user_active_time_3days.take(3)[0])

5000
('8613851429132', ['14', '14', '14', '14', '14', '14', '14', '11', '11', '11', '11', '11', '11', '14', '12', '12', '14', '14', '14', '14', '12', '12', '11', '11', '11', '11', '20', '20', '20', '20', '20', '20', '20', '20', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19'])


In [76]:
from collections import Counter
user_active_time_3days_hour = user_active_time_3days.map(lambda x: (x[0], dict(Counter(x[1]))))

In [77]:
print(user_active_time_3days_hour.take(3))

[('8613851429132', {'12': 4, '20': 8, '14': 12, '11': 10, '19': 39}), ('8615722826043', {'10': 1, '12': 1, '22': 2, '21': 20, '23': 3, '20': 32, '16': 3, '09': 1, '15': 5, '07': 20}), ('8615105191420', {'10': 10, '18': 2, '17': 6, '08': 30, '07': 24, '12': 1, '23': 36, '22': 13, '19': 1, '20': 13, '11': 26, '09': 25, '21': 62})]


In [78]:
def time_feature(time_dict):
    res = [0 for i in range(24)]
    time_hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
    for i in range(24):
        res[i] = time_dict.get(time_hour[i],0)
    return res

In [79]:
# user 的活跃时间。每行有24维，代表0-23每个小时的访问次数。
user_active_time_3days_hour_list = user_active_time_3days_hour.map(lambda x: (x[0], time_feature(x[1])))

In [80]:
print(user_active_time_3days_hour_list.take(1))

[('8613851429132', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 4, 0, 12, 0, 0, 0, 0, 39, 8, 0, 0, 0])]


In [82]:
# 用户前三天每个小时的访问qq的次数总和.
# [('8613851429132', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 4, 0, 12, 0, 0, 0, 0, 39, 8, 0, 0, 0])]
user_active_time_3days_hour_list.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_activetime")

### user_in_degree

In [7]:
# 三天连接qq服务器的总次数。in_degree or out_degree
# ('8613851429132', 73)
user_in_degree_3days = qq_5000_user_dpi.map(lambda x: (eval(x)[1], eval(x)[2])).groupByKey().mapValues(list).map(lambda x:(x[0],len(x[1])))

In [6]:
print(user_in_degree_3days.first())

('8613851429132', 73)


In [8]:
user_in_degree_3days.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_indegree")

### user_in_strength

In [9]:
# 用户三天对qq服务器的下行报文总数。
user_in_strength = qq_5000_user_dpi.map(lambda x: (eval(x)[1], eval(x)[9])).groupByKey().mapValues(list)

In [19]:
# ('8613851429132', 328)
user_in_strength = user_in_strength.map(lambda x:(x[0],sum([int(i) for i in x[1]])))

In [20]:
print(user_in_strength.take(1))

[('8613851429132', 328)]


In [24]:
user_in_strength.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_in_strength")

### user_out_strength

In [21]:
# 用户三天对qq服务器的上行报文总数。
user_out_strength = qq_5000_user_dpi.map(lambda x: (eval(x)[1], eval(x)[8])).groupByKey().mapValues(list)

In [22]:
# ('8613851429132', 331)
user_out_strength = user_out_strength.map(lambda x:(x[0],sum([int(i) for i in x[1]])))

In [23]:
print(user_out_strength.take(1))

[('8613851429132', 331)]


In [25]:
user_out_strength.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_out_strength")

## qq_server_node

### server-category

In [11]:
# server 提供的业务类型 ## 目标服务器只存在两种业务类型：'wap2', 'web'。不存在同时提供两种业务的ip.
# ('14.17.19.77', {'web'})
server_category = qq_5000_user_dpi.map(lambda x:(eval(x)[2], eval(x)[3])).groupByKey().mapValues(set)

In [12]:
print(server_category.first())
print(server_category.count())

('14.17.19.77', {'web'})
1684


In [13]:
set(server_category.map(lambda x: list(x[1])[0]).collect())

{'wap2', 'web'}

In [15]:
# ('14.17.19.77', 'web')
server_category = server_category.map(lambda x: (x[0], list(x[1])[0]))

In [16]:
server_category.take(3)

[('14.17.19.77', 'web'), ('111.10.27.180', 'web'), ('111.30.132.109', 'web')]

In [17]:
# ####只有一个是'wap2'，其他都是'web'。 ('10.0.0.172', 'wap2')#####
server_category.filter(lambda x: x[1] == 'wap2').collect()

[('10.0.0.172', 'wap2')]

In [18]:
# ('14.17.19.77', 'web')
server_category.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_category")

### server-service

In [19]:
# 大类业务ID
server_service_ID = qq_5000_user_dpi.map(lambda x:(eval(x)[2], eval(x)[10])).groupByKey().mapValues(set).map(lambda x:(x[0],list(x[1])))

In [20]:
server_service_ID.take(3)

[('14.17.19.77', ['4', '7']),
 ('111.10.27.180', ['4', '6']),
 ('111.30.132.109', ['4', '6'])]

In [21]:
server_service_ID.count()

1684

In [23]:
# 大类业务ID ('14.17.19.77', ['4', '7'])
server_service_ID.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_service_ID")

### server-sub_service

In [24]:
# 小类业务ID
server_service_id = qq_5000_user_dpi.map(lambda x:(eval(x)[2], eval(x)[11])).groupByKey().mapValues(set).map(lambda x:(x[0],list(x[1])))

In [25]:
server_service_id.take(3)

[('14.17.19.77', ['1025', '1793']),
 ('111.10.27.180', ['1025', '1029']),
 ('111.30.132.109', ['1025', '1029'])]

In [27]:
# 小类业务ID ('14.17.19.77', ['1025', '1793'])
server_service_id.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_service_subID")

### server_active_time

In [26]:
server_active_time_3days = qq_5000_user_dpi.map(lambda x: (eval(x)[2],eval(x)[0][-8:-6])).groupByKey().mapValues(list)

In [31]:
from collections import Counter
server_active_time_3days_hour = server_active_time_3days.map(lambda x: (x[0], dict(Counter(x[1]))))

In [32]:
print(server_active_time_3days_hour.first())

('14.17.19.77', {'10': 44, '18': 36, '14': 35, '13': 38, '21': 59, '01': 12, '17': 30, '05': 11, '22': 51, '04': 10, '16': 38, '11': 47, '03': 5, '00': 41, '08': 29, '07': 20, '12': 29, '23': 75, '19': 52, '20': 43, '02': 7, '06': 8, '09': 25, '15': 34})


In [66]:
# server 的活跃时间。每行有24维，代表0-23每个小时的访问次数。
server_active_time_3days_hour_list = server_active_time_3days_hour.map(lambda x: (x[0], time_feature(x[1])))

In [68]:
print(server_active_time_3days_hour_list.take(3))

[('14.17.19.77', [41, 12, 7, 5, 10, 11, 8, 20, 29, 25, 44, 47, 29, 38, 35, 34, 38, 30, 36, 52, 43, 59, 51, 75]), ('111.10.27.180', [1, 0, 2, 3, 3, 0, 1, 3, 4, 1, 6, 2, 1, 6, 2, 2, 2, 1, 7, 1, 0, 1, 4, 0]), ('111.30.132.109', [0, 0, 6, 12, 0, 4, 18, 35, 51, 130, 67, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0])]


In [71]:
server_active_time_3days_hour_list.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_active_time")

### server_in_degree

In [83]:
# 每个server三天总的访问次数.('14.17.19.77', 779)
server_in_degree = qq_5000_user_dpi.map(lambda x:(eval(x)[2], 1)).groupByKey().mapValues(len)

In [84]:
server_in_degree.take(1)

[('14.17.19.77', 779)]

In [85]:
# ('14.17.19.77', 779)
server_in_degree.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_indegree")

### server_in_strength

In [86]:
# qq服务器三天的下行报文总数。
server_in_strength = qq_5000_user_dpi.map(lambda x: (eval(x)[2], eval(x)[9])).groupByKey().mapValues(list)

In [89]:
server_in_strength = server_in_strength.map(lambda x:(x[0],sum([int(i) for i in x[1]])))

In [90]:
print(server_in_strength.take(2))

[('14.17.19.77', 4569), ('111.10.27.180', 4519)]


In [91]:
server_in_strength.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_in_strength")

### server_out_strength

In [92]:
# qq服务器三天的下行报文总数。
server_out_strength = qq_5000_user_dpi.map(lambda x: (eval(x)[2], eval(x)[8])).groupByKey().mapValues(list)

In [93]:
server_out_strength = server_out_strength.map(lambda x:(x[0],sum([int(i) for i in x[1]])))

In [95]:
print(server_out_strength.take(2))

[('14.17.19.77', 5792), ('111.10.27.180', 3065)]


In [96]:
server_out_strength.coalesce(1).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_out_strength")

## edge:day-user-server

#### day1: user-server: duration, uppacket, downpacket

In [86]:
# 连接时长7，上行报文8，下行报文9
from operator import add
day1_user_server_duration = qq_5000_user_dpi_day1.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[7])))\
.reduceByKey(add)
print(day1_user_server_duration.count())
print(day1_user_server_duration.take(3))
day1_user_server_duration.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_duration/1228")

76569
[('8615005141602-120.204.200.102', 25747), ('8618795963348-183.207.234.71', 37174), ('8615250987989-211.136.233.104', 6388)]


In [87]:
day1_user_server_uppacket = qq_5000_user_dpi_day1.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[8])))\
.reduceByKey(add)
print(day1_user_server_uppacket.count())
print(day1_user_server_uppacket.take(3))
day1_user_server_uppacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_uppacket/1228")

76569
[('8615005141602-120.204.200.102', 44), ('8618795963348-183.207.234.71', 10), ('8615250987989-211.136.233.104', 14)]


In [88]:
day1_user_server_downpacket = qq_5000_user_dpi_day1.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[9])))\
.reduceByKey(add)
print(day1_user_server_downpacket.count())
print(day1_user_server_downpacket.take(3))
day1_user_server_downpacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_downpacket/1228")

76569
[('8615005141602-120.204.200.102', 40), ('8618795963348-183.207.234.71', 5), ('8615250987989-211.136.233.104', 9)]


### day2: user-server: duration, uppacket, downpacket

In [89]:
day2_user_server_duration = qq_5000_user_dpi_day2.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[7])))\
.reduceByKey(add)
print(day2_user_server_duration.count())
print(day2_user_server_duration.take(3))
day2_user_server_duration.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_duration/1229")

76809
[('8613470823518-111.30.130.119', 334), ('8618795963348-183.207.234.71', 3060), ('8615105146953-113.108.19.85', 4740)]


In [90]:
day2_user_server_uppacket = qq_5000_user_dpi_day2.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[8])))\
.reduceByKey(add)
print(day2_user_server_uppacket.count())
print(day2_user_server_uppacket.take(3))
day2_user_server_uppacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_uppacket/1229")

76809
[('8613470823518-111.30.130.119', 4), ('8618795963348-183.207.234.71', 19), ('8615105146953-113.108.19.85', 13)]


In [91]:
day2_user_server_downpacket = qq_5000_user_dpi_day2.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[9])))\
.reduceByKey(add)
print(day2_user_server_downpacket.count())
print(day2_user_server_downpacket.take(3))
day2_user_server_downpacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_downpacket/1229")

76809
[('8613470823518-111.30.130.119', 2), ('8618795963348-183.207.234.71', 18), ('8615105146953-113.108.19.85', 9)]


### day3: user-server: duration, uppacket, downpacket

In [92]:
day3_user_server_duration = qq_5000_user_dpi_day3.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[7])))\
.reduceByKey(add)
print(day3_user_server_duration.count())
print(day3_user_server_duration.take(3))
day3_user_server_duration.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_duration/1230")

82208
[('8613813067618-117.135.171.245', 2777), ('8618795929560-183.232.88.152', 109014), ('8618251911961-111.30.128.27', 7327)]


In [93]:
day3_user_server_uppacket = qq_5000_user_dpi_day3.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[8])))\
.reduceByKey(add)
print(day3_user_server_uppacket.count())
print(day3_user_server_uppacket.take(3))
day3_user_server_uppacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_uppacket/1230")

82208
[('8613813067618-117.135.171.245', 40), ('8618795929560-183.232.88.152', 153), ('8618251911961-111.30.128.27', 16)]


In [94]:
day3_user_server_downpacket = qq_5000_user_dpi_day3.map(lambda x:((eval(x)[1] + '-' + eval(x)[2]),int(eval(x)[9])))\
.reduceByKey(add)
print(day3_user_server_downpacket.count())
print(day3_user_server_downpacket.take(3))
day3_user_server_downpacket.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_downpacket/1230")

82208
[('8613813067618-117.135.171.245', 39), ('8618795929560-183.232.88.152', 144), ('8618251911961-111.30.128.27', 12)]


## edges

In [104]:
positive_edge_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/edge_downpacket/*/part-*"

In [109]:
# 所有有记录的边
all_postive_edges = sc.textFile(positive_edge_f).map(lambda x: eval(x)[0]).collect()

In [125]:
print(len(all_postive_edges))
print(all_postive_edges[:3])

235586
['8615150598197-120.204.200.174', '8613951990902-113.108.86.122', '8615861816974-183.207.233.95']


In [96]:
# 5000个用户的id和1684个服务器ip
user_phone_5000_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_phone_5000/part-*"
server_ip_1684_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_ip_1684/part-*"

In [100]:
user_phone_5000 = sc.textFile(user_phone_5000_f).collect()
server_ip_1684 = sc.textFile(server_ip_1684_f).collect()

In [101]:
print(len(user_phone_5000),len(server_ip_1684))

5000 1684


In [115]:
all_edges = []
for u in user_phone_5000:
    for s in server_ip_1684:
        edge = u + '-' + s
        all_edges.append(edge)

In [117]:
print(len(all_edges))
print(all_edges[:3])

8420000
['8615195898003-14.17.19.77', '8615195898003-111.10.27.180', '8615195898003-111.30.132.109']


In [121]:
unlabled_edges = list(set(all_edges)-set(all_postive_edges))

In [123]:
print(len(unlabled_edges))
print(unlabled_edges[:3])

8268837
['8615151840328-119.167.195.26', '8615850607902-203.205.161.165', '8615151857865-117.135.171.164']


In [ ]:
unlabled_edges = ['day123-' + i for i in unlabled_edges ]

In [ ]:
sc.parallelize(unlabled_edges).coalesce(24)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_unlabled_pairs")

### Test data

In [134]:
test_positive_pairs = qq_5000_user_dpi_day4.map(lambda x: eval(x)[1] + '-' + eval(x)[2])

In [136]:
test_unlabled_pairs = list(set(all_edges) - set(test_positive_pairs.collect()))

In [137]:
print(len(test_unlabled_pairs))

8342347


In [138]:
test_positive_pairs.coalesce(10)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/test_pos_pairs")

In [139]:
sc.parallelize(test_unlabled_pairs).coalesce(24)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/test_unlabled_pairs")